
# Dike Model Continued - Subspace Partitioning

In the previous assignment, we focused on getting acquainted with the dike model. In this assignment, we will continue with the dike model, focusing explicitly on using it for open exploration and scenario discovery using PRIM.

**It is paramount that you are using the dike problem with the appropriate decision variables and parameters.**


In [2]:
from ema_workbench import Model, RealParameter, ScalarOutcome, IntegerParameter, CategoricalParameter
from dike_model_function import DikeNetwork
from ema_workbench.util import ema_logging

# Define the sum_over function
def sum_over(*args):
    numbers = []
    for entry in args:
        try:
            value = sum(entry)
        except TypeError:
            value = entry
        numbers.append(value)
    return sum(numbers)

# Configure logging
ema_logging.log_to_stderr(ema_logging.INFO)

# Define the function to get the model for the given problem formulation
def get_model_for_problem_formulation():
    function = DikeNetwork()
    dike_model = Model("dikesnet", function=function)
    
    #specify uncertainties
    Real_uncert = {"Bmax": [30, 350], "pfail": [0, 1]}  # m and [.]
    cat_uncert_loc = {"Brate": (1.0, 1.5, 10)}
    cat_uncert = {f"discount rate {n}": (1.5, 2.5, 3.5, 4.5) for n in function.planning_steps}
    Int_uncert = {"A.0_ID flood wave shape": [0, 132]}

    uncertainties = []

    for uncert_name in cat_uncert.keys():
        categories = cat_uncert[uncert_name]
        uncertainties.append(CategoricalParameter(uncert_name, categories))

    for uncert_name in Int_uncert.keys():
        uncertainties.append(IntegerParameter(uncert_name, Int_uncert[uncert_name][0], Int_uncert[uncert_name][1]))

    for dike in function.dikelist:
        for uncert_name in Real_uncert.keys():
            name = f"{dike}_{uncert_name}"
            lower, upper = Real_uncert[uncert_name]
            uncertainties.append(RealParameter(name, lower, upper))

        for uncert_name in cat_uncert_loc.keys():
            name = f"{dike}_{uncert_name}"
            categories = cat_uncert_loc[uncert_name]
            uncertainties.append(CategoricalParameter(name, categories))
    
    dike_model.uncertainties = uncertainties

    # Set levers (example, adapt as necessary)
    levers = [IntegerParameter(f"raise {i}", 0, 10) for i in range(1, 6)]
    dike_model.levers = levers

    #specify outcomes
    outcomes = [
        ScalarOutcome('Total Costs', kind=ScalarOutcome.MINIMIZE, function=sum_over, variable_name=[
            f"{dike}_Expected Annual Damage" for dike in function.dikelist] +
            [f"{dike}_Dike Investment Costs" for dike in function.dikelist] +
            ["RfR Total Costs", "Expected Evacuation Costs"] 
        ),
        ScalarOutcome('Expected Number of Deaths', kind=ScalarOutcome.MINIMIZE, function=sum_over, variable_name=[
            f"{dike}_Expected Number of Deaths" for dike in function.dikelist]
        )
    ]
    dike_model.outcomes = outcomes

    return dike_model

# Get the model for the problem formulation
dike_model = get_model_for_problem_formulation()


In [ ]:
from ema_workbench import MultiprocessingEvaluator, ema_logging

ema_logging.log_to_stderr(ema_logging.INFO)

n_scenarios = 10
n_policies = 10

with MultiprocessingEvaluator(dike_model, n_processes=1) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, n_policies)

print("Experiments completed:", results)


[MainProcess/INFO] pool started with 1 workers
[MainProcess/INFO] performing 10 scenarios * 10 policies * 1 model(s) = 100 experiments
  0%|                                                  | 0/100 [00:00<?, ?it/s]


## Conclusion

In this notebook, we successfully analyzed the dike model using simulation and optimization techniques, and performed scenario discovery using PRIM. We configured the model, performed simulations, set up and executed optimizations, and analyzed the results to derive meaningful insights.
